In [21]:
#movie rec using collaborative data and matrix factorization
import csv
import string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk
from scipy import optimize
from sklearn.decomposition import TruncatedSVD

#some code from http://alexabate.github.io/2016/11/05/movie-lens.html
#some code from https://nikhilwins.wordpress.com/2015/09/18/movie-recommendations-how-does-netflix-do-it-a-9-step-coding-intuitive-guide-into-collaborative-filtering/

user_item_ratings = pd.read_csv('ratings.csv', sep=',')
#user_item_ratings.head()
number_of_unique_users = len(user_item_ratings['userId'].unique())
number_of_unique_movies = len(user_item_ratings['movieId'].unique())
number_of_ratings = len(user_item_ratings)
#print user_item_ratings
print number_of_unique_users
print number_of_unique_movies

ratingsMatrix=pd.pivot_table(user_item_ratings, values='rating', fill_value=0, columns='userId', index='movieId')#
#print ratingsMatrix.head

#ratingsMatrix=ratingsMatrix.iloc[:100,:]

print ratingsMatrix.shape

671
9066
(9066, 671)


In [22]:
did_rate = (ratingsMatrix != 0) * 1
print did_rate.shape
#idx = np.where(did_rate.loc[30,:] ==1)[0]
#print did_rate[10,:]
#idx = np.where(did_rate.iloc[1,:] ==1)[0]
#print idx
#print ratingsMatrix.iloc[1,idx]
#print np.mean(ratingsMatrix.loc[1,idx+1])

(9066, 671)


In [23]:
#n_movies=number_of_unique_movies
n_movies=ratingsMatrix.shape[0]
n_users=number_of_unique_users


#normalize_ratings from https://nikhilwins.wordpress.com/2015/09/18/movie-recommendations-how-does-netflix-do-it-a-9-step-coding-intuitive-guide-into-collaborative-filtering/
def normalize_ratings(ratings, did_rate):
    num_movies = ratings.shape[0]
    ratings_mean = np.zeros(shape = (num_movies, 1))
    ratings_norm = np.zeros(shape = ratings.shape)
    
    for i in range(0, num_movies):
        # Get all the indexes where there is a 1
        idx = np.where(did_rate.iloc[i,:] ==1)[0]
        # Calculate mean rating of ith movie only from users that gave a rating
        ratings_mean[i] = np.mean(ratings.iloc[i, idx])
         #ratings_mean[i]
        ratings_norm[i, idx] = ratings.iloc[i, idx] - ratings_mean[i]
        #(EKC: make sure these indices aren't off by one)
        #print i
    return (ratings_norm, ratings_mean)


ratings_norm, ratings_mean = normalize_ratings(ratingsMatrix, did_rate)


In [24]:
# code for gradient descent from https://nikhilwins.wordpress.com/2015/09/18/movie-recommendations-how-does-netflix-do-it-a-9-step-coding-intuitive-guide-into-collaborative-filtering/


def unroll_params(X_and_theta, num_users, num_movies, num_features):
    # Retrieve the X and theta matrixes from X_and_theta, based on their dimensions (num_features, num_movies, num_movies)
    # --------------------------------------------------------------------------------------------------------------
    # Get the first (movies * features) rows in the column vector
    first_30 = X_and_theta[:num_movies * num_features]
    #(EKC--doesn't this give you the last 30 not the first? does it have to do with the way things are flattened?)
    # Reshape this column vector into a matrix
    X = first_30.reshape((num_features, num_movies)).transpose()
    # Get the rest of the numbers
    last_18 = X_and_theta[num_movies * num_features:]
    # Reshape this column vector into a 6 X 3 matrix
    theta = last_18.reshape(num_features, num_users ).transpose()
    return X, theta




def calculate_gradient(X_and_theta, ratings, did_rate, num_users, num_movies, num_features, reg_param):
    X, theta = unroll_params(X_and_theta, num_users, num_movies, num_features)
    
    # we multiply by did_rate because we only want to consider observations for which a rating was given
    difference = X.dot( theta.T ) * did_rate - ratings
    X_grad = difference.dot( theta ) + reg_param * X
    theta_grad = difference.T.dot( X ) + reg_param * theta
    
    # wrap the gradients back into a column vector 
    return np.concatenate((X_init.T.flatten(), B_init.T.flatten()))




def calculate_cost(X_and_theta, ratings, did_rate, num_users, num_movies, num_features, reg_param):
    X, theta = unroll_params(X_and_theta, num_users, num_movies, num_features)
    
    # we multiply (element-wise) by did_rate because we only want to consider observations for which a rating was given
    cost = np.sum( (X.dot( theta.T ) * did_rate - ratings) ** 2 ) / 2
    # '**' means an element-wise power
    regularization = (reg_param / 2) * (np.sum( theta**2 ) + np.sum(X**2))
    #(EKC--changed the above from sum to np.sum)
    return cost + regularization

In [25]:
did_rate=did_rate.as_matrix()

In [26]:
#use normalized ratings
#actually use random ratings in a small matrix first, to test

n_features=3

n_users=number_of_unique_users
n_movies=ratingsMatrix.shape[0]
#n_movies=number_of_unique_movies
#n_users=3
#n_movies=4
print n_users
print n_movies

X_init=np.random.rand(n_movies, n_features)
B_init=np.random.rand(n_features, n_users)
#print B_init
predicted_ratings=X_init.dot(B_init) 

fake_ratings=np.random.rand(n_movies, n_users)
fake_did_rate = (fake_ratings != 0) * 1


#print ratings_norm.shape
#print fake_ratings.shape

#(the next dozen lines or so just test some of the methods)

initial_X_and_B = np.concatenate((X_init.T.flatten(), B_init.T.flatten()))
print initial_X_and_B.shape

reg_param=1.0 #(EKC: adjust as needed)

X, B = unroll_params(initial_X_and_B, n_users, n_movies, n_features)
#EKC - checked that these come back out the right way, using small random test arrays. 
print X.shape
print B.shape

difference=X.dot( B.T ) * did_rate - ratings_norm
print difference.shape
cost=np.sum(( difference) ** 2 ) / 2
print cost


671
9066
(29211,)
(9066, 3)
(671, 3)
(9066, 671)
72954.3948964


In [27]:

#minimized_cost_and_optimal_params = optimize.fmin_cg(calculate_cost, fprime=calculate_gradient, x0=initial_X_and_B, args=(ratings_norm, did_rate, n_users, n_movies, n_features, reg_param), maxiter=100, disp=True, full_output=True ) 

#(something is going on here, it never iterates. maybe an issue with step size?)


In [34]:

#grad descent for cost
#some code for computing cost and gradient from https://nikhilwins.wordpress.com/2015/09/18/movie-recommendations-how-does-netflix-do-it-a-9-step-coding-intuitive-guide-into-collaborative-filtering/

#this one iterates but crashes or runs out of memory 

X=X_init
B=B_init
    
difference=X.dot( B ) * did_rate - ratingsMatrix.as_matrix()
regularization= (reg_param / 2) * (np.sum( B_init**2 ) + np.sum(X_init**2))
cost=np.sum(( difference ) ** 2 ) / 2 + regularization

alpha=.0007 #something close to this value works for alpha; .001 is too big.
iter=0

#there should probably be other stopping criteria: e.g. if the gradient (not the cost) is small
#also if the cost isn't consistently decreasing (probably meaning alpha is too big)
while cost>100 and iter<20:   
    # we calculate the gradients (derivatives) of the cost with respect to X and theta
    X_grad = difference.dot( B.T ) + reg_param * X
    B_grad = difference.T.dot( X ).T + reg_param * B
    # (if there's a problem here something probably just has to be transposed--check that it's still right.)
    
    X=X-X_grad*alpha
    B=B-B_grad*alpha #(same alpha?)
    difference=X.dot( B ) * did_rate - ratingsMatrix.as_matrix()
    cost=np.sum(( difference ) ** 2 ) / 2 + regularization
    iter=iter+1
    print cost
print iter


5711163.55612
243180819047.0
2.73587644238e+25
4.39872152003e+67
1.94082780539e+194
inf
nan
7


In [10]:

# use (leave-1-or-some-out) cross-validation - split into test and train sets 
# and only train the model based on the training set--create did_rate_train, which has zeros in all the rows
# representing the test users, so that error in those predictions does not contribute to the cost, i.e., doesn't
# train the model.
#(also put zeros in y_train? or no?)